<a href="https://colab.research.google.com/github/vannissimo/snake-ga/blob/master/pitch_control_acm_marcature2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
import numpy as np
from bisect import bisect
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import concat,col,lit,desc
from operator import itemgetter
import itertools 
from bisect import bisect
import pandas as pd

In [ ]:
team = team_a = 'milan'
opponent = 'cagliari'
half = 'second half'
half_split = '2'
player_in_possess = 'all'
secondi_di_differenza_giocatori_eventi = {'cagliari': -2}

name_path = team +"_"+str(opponent)+"_"+str(half).replace(' ','_')+"_"+str(half_split)+"_"+player_in_possess

name_path

'milan_cagliari_second_half_2_all'

In [ ]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading data as parquet
pre_data_players = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/frames_milan_cagliari.parquet").drop('index')

pre_pre_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/' + 'pitch_control_' + name_path + '.csv',header='True')
pre_pre_df = pre_pre_df.withColumn('x', pre_pre_df['x'].cast(DoubleType())).withColumn('y', pre_pre_df['y'].cast(DoubleType())).withColumn('target_y', pre_pre_df['target_y'].cast(DoubleType())).withColumn('target_x', pre_pre_df['target_x'].cast(DoubleType()))

pre_pre_df.where(pre_pre_df['target_x'] > -999).where(pre_pre_df['target_y'] < 9999).groupBy('time').agg(func.min('target_x'),func.max('target_x'),func.min('target_y'),func.max('target_y')).show()

+------+-------------+-------------+-------------+-------------+
|  time|min(target_x)|max(target_x)|min(target_y)|max(target_y)|
+------+-------------+-------------+-------------+-------------+
|4315.0|        -48.0|         54.0|        -35.0|         36.0|
+------+-------------+-------------+-------------+-------------+



In [ ]:
#create parameters input boxes (Databricks)

if opponent == 'milan':
  opponent_f = team_a
else:
  opponent_f = opponent

In [ ]:
from pyspark.sql import functions as func

pre_data_players = pre_data_players.withColumnRenamed('team_name','team').withColumnRenamed('position','role')

min_half_time, max_half_time = pre_data_players.where(pre_data_players['half'] == half).groupBy('half').agg(func.min('time'),func.max('time')).collect()[0][1:]

if half_split == '1':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time) & (pre_data_players['time'] <= min_half_time+float((max_half_time-min_half_time)/2.0)))
  
elif half_split == '2':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time+float((max_half_time-min_half_time)/2.0)) & (pre_data_players['time'] < max_half_time))


In [ ]:
data_players.groupBy('time').count().show()

+------+-----+
|  time|count|
+------+-----+
|5601.6|   23|
|5603.2|   23|
|5617.4|   23|
|4238.2|   23|
|4247.6|   23|
|4262.6|   23|
|4271.0|   23|
|4298.7|   23|
|4320.6|   23|
|4361.6|   23|
|4393.0|   23|
|4446.0|   23|
|4491.3|   23|
|4542.6|   23|
|4550.9|   23|
|4560.5|   23|
|4566.2|   23|
|4580.1|   23|
|4618.0|   23|
|4625.7|   23|
+------+-----+
only showing top 20 rows



In [ ]:
from operator import itemgetter
from pyspark.sql import functions as func
from pyspark.sql.functions import lit

import itertools 
import bisect
import numpy as np
from pyspark.sql.types import DoubleType
import pandas as pd

team_a_lower = team_a.lower()
granularity = 0.5
max_ball_distance = 70.0
max_meters_distance_player_can_receive_ball = 30.0
opponent_lower = opponent.lower()

team_a_goalies = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').select('player_id').distinct().collect()[0]['player_id']
opponent_goalies = data_players.where(data_players['team'] == opponent.title()).where(data_players['role'] == 'Goalkeeper').select('player_id').distinct().collect()[0]['player_id']

goalie_x = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').groupBy('role').avg('x').rdd.max()[1]

team_a_goalies,opponent_goalies,goalie_x

('p129487', 'p213535', -35.384379295644386)

In [ ]:
min_x = data_players.select('x').rdd.min()[0]
max_x = data_players.select('x').rdd.max()[0]
min_y = data_players.select('y').rdd.min()[0]
max_y = data_players.select('y').rdd.max()[0]

x_lines = list(np.arange(int(min_x)-1,int(max_x)+1,float(max_x-min_x)/6.0))
y_lines = list(np.arange(int(min_y)-1,int(max_y)+1,float(max_y-min_y)/4.0))

mid_x, mid_y = x_lines[3], float(y_lines[2])

if float(np.abs(goalie_x - max_x)) < float(np.abs(goalie_x - min_x)):
  verse = 'right_to_left'
  opponent_verse = 'left_to_right'
else:
  verse = 'left_to_right'
  opponent_verse = 'right_to_left'

team_a_goalies,verse

('p129487', 'left_to_right')

In [ ]:
pre_pre_df = pre_pre_df.withColumn('time', pre_pre_df['time'].cast(DoubleType()))

In [ ]:
possess = pre_pre_df.where(pre_pre_df['target_x'] < -950).select('player_id','team','time','x','y').distinct().withColumnRenamed('player_id','player_in_possess').withColumnRenamed('team','team_in_possess')

timings = sorted(possess.select('time').rdd.map(lambda a : a[0]).collect())
timings

[4315.0]

In [ ]:
df = pre_pre_df.where(pre_pre_df['target_x'] > -999).distinct()

In [ ]:
#offside_players = pre_df.where(pre_df['team'] == team_a.capitalize()).select('player_id','time','offside_flag').distinct()

In [ ]:
verse

'left_to_right'

In [ ]:
df_locs = df.select('player_id','time','team','x','y').distinct()

if verse == 'right_to_left':
  opponent_goal_line = [min_x+5.0, 0.0]  
  team_a_goal_line = [max_x-5.0, 0.0]
else:
  opponent_goal_line = [max_x-5.0, 0.0]
  team_a_goal_line = [min_x+5.0, 0.0]
  
df_locs = df_locs.withColumn('distance_from_opponent_goal_line', 0.25*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2 + (df_locs['y'] - opponent_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2))
df_locs = df_locs.withColumn('distance_from_team_goal_line', 0.25*func.sqrt((df_locs['x'] - team_a_goal_line[0])**2 + (df_locs['y'] - team_a_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - team_a_goal_line[0])**2))

panda_locs = df_locs.toPandas()

verse,opponent_goal_line

('left_to_right', [59.0, 0.0])

In [ ]:
x_ranges = list(np.arange(min_x,max_x,3.0))
y_ranges = list(np.arange(min_y,max_y,3.0))

In [ ]:
x_ranges, y_ranges

([-60.20000076293945,
  -57.20000076293945,
  -54.20000076293945,
  -51.20000076293945,
  -48.20000076293945,
  -45.20000076293945,
  -42.20000076293945,
  -39.20000076293945,
  -36.20000076293945,
  -33.20000076293945,
  -30.200000762939453,
  -27.200000762939453,
  -24.200000762939453,
  -21.200000762939453,
  -18.200000762939453,
  -15.200000762939453,
  -12.200000762939453,
  -9.200000762939453,
  -6.200000762939453,
  -3.200000762939453,
  -0.20000076293945312,
  2.799999237060547,
  5.799999237060547,
  8.799999237060547,
  11.799999237060547,
  14.799999237060547,
  17.799999237060547,
  20.799999237060547,
  23.799999237060547,
  26.799999237060547,
  29.799999237060547,
  32.79999923706055,
  35.79999923706055,
  38.79999923706055,
  41.79999923706055,
  44.79999923706055,
  47.79999923706055,
  50.79999923706055,
  53.79999923706055,
  56.79999923706055,
  59.79999923706055,
  62.79999923706055],
 [-49.29999923706055,
  -46.29999923706055,
  -43.29999923706055,
  -40.29999923

In [ ]:
def x_area_handling(x):
  
  if x >= 30:
    return x
  elif x < 30 and x >= 10:
    return int(0.8*float(x))
  elif x < 10 and x >= -20:
    return int(0.5*float(x))
  elif x < -20:
    return 10

x_single_area_prob = 100.0/float(len(x_ranges))
y_single_area_prob = 100.0/float(len(y_ranges))

x_ranges_value_right_to_left = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(len(x_ranges)-x_ranges.index(d))), x_ranges))
x_ranges_value_left_to_right = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(x_ranges.index(d))), x_ranges))
y_ranges_value = list(map(lambda d : 100-1.75*(float(y_single_area_prob)*float(np.abs((mid_y-y_ranges.index(d))))), y_ranges))
                      
if verse == 'left_to_right':
  x_ranges_value = x_ranges_value_left_to_right
  opp_x_ranges_value = x_ranges_value_right_to_left
elif verse == 'right_to_left':
  x_ranges_value = x_ranges_value_right_to_left
  opp_x_ranges_value = x_ranges_value_left_to_right
  
x_ranges_value,y_ranges_value

([0,
  1,
  2,
  3,
  4,
  9,
  11,
  13,
  15,
  17,
  19,
  20,
  22,
  30.952380952380953,
  33.333333333333336,
  35.714285714285715,
  38.095238095238095,
  40.476190476190474,
  42.857142857142854,
  45.23809523809524,
  47.61904761904762,
  50.0,
  52.38095238095238,
  54.76190476190476,
  57.14285714285714,
  59.523809523809526,
  61.904761904761905,
  64.28571428571429,
  66.66666666666667,
  69.04761904761905,
  71.42857142857143,
  73.80952380952381,
  76.19047619047619,
  78.57142857142857,
  80.95238095238095,
  83.33333333333333,
  85.71428571428571,
  88.09523809523809,
  90.47619047619048,
  92.85714285714286,
  95.23809523809524,
  97.61904761904762],
 [57.15517011182062,
  51.12068735319993,
  45.086204594579236,
  39.051721835958546,
  33.01723907733785,
  26.98275631871718,
  20.94827356009648,
  14.913790801475798,
  8.8793080428551,
  2.8448252842344175,
  -3.189657474386266,
  -9.224140233006963,
  -15.258622991627647,
  -21.293105750248344,
  -27.32758850886904,

In [ ]:
def probabilityPlusMinus(x):
  
  if x[6] == team_a.title():
    return list(x[:3])+[-float(x[3])]+list(x[4:])+[opponent_verse]
  else:
    return list(x)+[verse]
  
def opponentCount(x):
  
  try:
    team_1 = float(x[3])
  except:
    team_1 = 0.0

  try:
    team_2 = float(x[5])
  except:
    team_2 = 0.0

  try:
    opp_1 = int(x[4])
  except:
    opp_1 = 0

  try:
    opp_2 = int(x[6])
  except:
    opp_2 = 0

  if team_1 > team_2:
    return [x[0],x[1],x[2],x[3],-opp_2]
  else:
    return [x[0],x[1],x[2],x[5],-opp_1]

df = df.withColumn('probability', df['probability'].cast(DoubleType()))

pre_max_pitch_pivot_gr = df.where(df['team'] != '').groupBy('target_x','target_y','time').pivot('team').agg(func.max('probability'),func.count('player_id'))

max_pitch_pivot = spark.createDataFrame(pre_max_pitch_pivot_gr.rdd.map(lambda a : opponentCount(a)),['target_x','target_y','time','probability','opponents_count'])

max_pitch_pivot.show()

+--------+--------+------+------------------+---------------+
|target_x|target_y|  time|       probability|opponents_count|
+--------+--------+------+------------------+---------------+
|    17.0|    29.0|4315.0|57.971887157093406|             -1|
|    21.0|   -17.0|4315.0|63.175744206576454|             -1|
|    35.0|    36.0|4315.0|63.394528650866064|             -1|
|    48.0|     8.0|4315.0|54.120299248096224|              0|
|    45.0|    20.0|4315.0| 58.33878557297363|              0|
|   -39.0|     7.0|4315.0|             100.0|              0|
|    -7.0|   -24.0|4315.0| 63.62808998150003|             -1|
|    49.0|   -25.0|4315.0| 65.09099107915893|              0|
|    48.0|   -23.0|4315.0|             100.0|              0|
|    45.0|    18.0|4315.0|52.522875343125875|              0|
|    53.0|     2.0|4315.0|             100.0|              0|
|   -12.0|     0.0|4315.0| 64.15895440209394|              0|
|   -33.0|    27.0|4315.0|             100.0|              0|
|    -6.

In [ ]:
max_player = df.select('target_x','target_y','time','player_id','team','probability','distance_from_target','x','y','slope_fuzzy_compare','ball_distance')

In [ ]:
max_pitch_pivot_j = max_pitch_pivot.join(max_player, ['target_x','target_y','time','probability'])

max_pitch_pivot_jj = spark.createDataFrame(max_pitch_pivot_j.rdd.map(lambda f : probabilityPlusMinus(f)),max_pitch_pivot_j.columns+['verse'])

pre_max_pitch_pivot_jjj = max_pitch_pivot_jj.where(max_pitch_pivot_jj['team'] == opponent.capitalize())

pre_max_pitch_pivot_jjj.show()

+--------+--------+------+------------------+---------------+---------+--------+--------------------+------------------+-------------------+-------------------+------------------+-------------+
|target_x|target_y|  time|       probability|opponents_count|player_id|    team|distance_from_target|                 x|                  y|slope_fuzzy_compare|     ball_distance|        verse|
+--------+--------+------+------------------+---------------+---------+--------+--------------------+------------------+-------------------+-------------------+------------------+-------------+
|    35.0|    36.0|4315.0|63.394528650866064|             -1|  Dalbert|Cagliari|  17.577687702584534| 32.42937088012695| 18.611297607421875|                3.0|16.574581114380802|left_to_right|
|    48.0|     8.0|4315.0|54.120299248096224|              0|  Carboni|Cagliari|   9.143655016386232|38.900001525878906| 7.1075568199157715|                8.0| 39.77402009070472|left_to_right|
|    45.0|    20.0|4315.0| 58.

In [ ]:
max_pitch_pivot_jj.groupBy('player_id','team').avg('probability').show()

+---------+--------+------------------+
|player_id|    team|  avg(probability)|
+---------+--------+------------------+
| Florenzi|   Milan|-53.90035435104492|
|   Deiola|Cagliari|46.397150313647835|
|    Kjaer|   Milan|-71.98029489568307|
| Calabria|   Milan|-88.25131046358727|
|    Marin|Cagliari| 56.39003581379009|
|Strootman|Cagliari| 60.46467379297676|
|   Giroud|   Milan| -73.6283640939823|
|    Zappa|Cagliari|55.023978631277124|
|     Diaz|   Milan|-43.93988742990383|
|    Rebic|   Milan|-70.23229383205116|
| Bennacer|   Milan|-72.97034582147239|
|Pavoletti|Cagliari| 66.79955822292766|
|  Carboni|Cagliari| 76.96023260944897|
|   Krunic|   Milan|-65.01865090837212|
|    Godin|Cagliari| 84.43579298814518|
|   Tomori|   Milan|-55.08529392808189|
|  Dalbert|Cagliari| 77.76126696407854|
|  Maignan|   Milan|-95.76869437192407|
|   Nandez|Cagliari| 63.40997697131176|
|Radunovic|Cagliari| 83.57079808946607|
+---------+--------+------------------+
only showing top 20 rows



In [ ]:
pd_probs = pre_max_pitch_pivot_jjj.toPandas()

In [ ]:
def findBestMaxObs(x,panda,max_obs):
  
  probs = panda[panda['player_id'] == x[0]][panda['time'] == x[1]][['probability','distance_from_target']].values.tolist()
  
  try:
    first_twenty_probs = sorted(list(map(lambda a : a[0],probs)), reverse = True)[:max_obs]
    prob_twentieth = first_twenty_probs[-1]
    distance_twentieth = 100.0

    if float(prob_twentieth) == 100.0:
      first_twenty_distances = sorted(list(map(lambda g : g[1], list(filter(lambda a : a[0] == 100.0,probs)))))[:max_obs]
      distance_twentieth = first_twenty_distances[-1]

    return [prob_twentieth,distance_twentieth]
  except:
    return [0.0,0.0]

players_times_values = spark.createDataFrame(df.where(df['team'] == opponent.title()).select('player_id','time').distinct().rdd.map(lambda a : list(a)+findBestMaxObs(a,pd_probs,60)),['player_id','time','probability_twentieth','distance_twentieth'])

players_times_values.show()

+---------+------+---------------------+------------------+
|player_id|  time|probability_twentieth|distance_twentieth|
+---------+------+---------------------+------------------+
|   Deiola|4315.0|    33.03402394160351|             100.0|
|  Dalbert|4315.0|                100.0|              null|
|    Zappa|4315.0|    56.27228870597062|             100.0|
|    Godin|4315.0|                100.0|              null|
|   Nandez|4315.0|    44.13238787497912|             100.0|
|Radunovic|4315.0|                100.0|              null|
|Pavoletti|4315.0|                100.0|              null|
|    Marin|4315.0|   43.667234253131184|             100.0|
|  Carboni|4315.0|                100.0|              null|
|Strootman|4315.0|   55.735133150829576|             100.0|
+---------+------+---------------------+------------------+



In [ ]:
pd_to_filter = players_times_values.toPandas()

In [ ]:
pre_max_pitch_pivot_jjj

DataFrame[target_x: double, target_y: double, time: double, probability: double, opponents_count: bigint, player_id: string, team: string, distance_from_target: string, x: double, y: double, slope_fuzzy_compare: string, ball_distance: string, verse: string]

In [ ]:
def obsToFilter(x, panda):
  
  prob_filter, distance_filter = panda[panda['player_id'] == x[5]][panda['time'] == x[2]][['probability_twentieth','distance_twentieth']].values.tolist()[0]
  
  flag = ''
  
  if (float(x[3]) < prob_filter) or (float(x[7]) > distance_filter):
    flag = 'out'
  
  return flag
  
max_pitch_pivot_jjj_pre_filt = spark.createDataFrame(pre_max_pitch_pivot_jjj.rdd.map(lambda a : list(a)+[obsToFilter(a, pd_to_filter)]), pre_max_pitch_pivot_jjj.columns+['obs_flag'])

max_pitch_pivot_jjj = max_pitch_pivot_jjj_pre_filt.where(max_pitch_pivot_jjj_pre_filt['obs_flag'] == '').where(max_pitch_pivot_jjj_pre_filt['probability'] > 0).drop('obs_flag')

In [ ]:
max_pitch_pivot_jjj.show()

+--------+--------+------+-----------------+---------------+---------+--------+--------------------+------------------+-------------------+-------------------+------------------+-------------+
|target_x|target_y|  time|      probability|opponents_count|player_id|    team|distance_from_target|                 x|                  y|slope_fuzzy_compare|     ball_distance|        verse|
+--------+--------+------+-----------------+---------------+---------+--------+--------------------+------------------+-------------------+-------------------+------------------+-------------+
|    48.0|   -23.0|4315.0|            100.0|              0|    Godin|Cagliari|  11.425930527775757|44.627525329589844|-12.083118438720703|                5.0| 64.77379564512465|left_to_right|
|    53.0|     2.0|4315.0|            100.0|              0|Radunovic|Cagliari|   6.851919093555038| 47.80747985839844|-2.4706296920776367|                4.0| 47.51888740553346|left_to_right|
|    21.0|   -12.0|4315.0|         

In [ ]:
#display(max_pitch_pivot_jjj.where(max_pitch_pivot_jjj['target_x'] == 42.0).where(max_pitch_pivot_jjj['target_y'] == 5.0).where(max_pitch_pivot_jjj['time'] == 1135.0))

In [ ]:
from pyspark.sql.functions import lit

def ifHigherThanLimit(a,limit,maxMinFlag): 
  if maxMinFlag == 'max':
    return float(np.min([limit,a]))
  elif maxMinFlag == 'min':
    return float(np.max([limit,a]))

def addFakeProb(team):
  
  if team == opponent.title():
    return 150.0
  else:
    return -150.0

  
### PITCH ###
pitch_pivot = max_pitch_pivot_jj.select('target_x','target_y','time','probability','player_id','team')
pitch_pivot = pitch_pivot.withColumn('type',lit('pitch'))
pitch_pivot = pitch_pivot.withColumnRenamed('target_x','x').withColumnRenamed('target_y','y')

### PLAYERS ###
players_locs = spark.createDataFrame(data_players.select('player_id','team','time','x','y').withColumn('type',lit('player')).distinct().rdd.map(lambda a : list(a)+[addFakeProb(a[1])]), ['player_id','team','time','x','y','type','probability'])
players_locs = players_locs.where(players_locs['time'].isin(timings))

### BALL ###
ball = players_locs.where(players_locs['player_id'] == 'ball').orderBy('time')
ball_locs = ball.withColumn('probability',lit(200.0)).withColumn('type',lit('ball')).select('x','y','time','probability','player_id','team','type')
pd_ball = ball.toPandas()

In [ ]:
players_ids = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/OPTA/opta_seriea_players.csv", header='True')
players_ids.show()

+---------+------------------+-----------+----------+-----------+----------+--------------------+----------+--------------+--------------------+----------+------------------+------+------+-------+----------+-----------------+--------------+
|player_id|     real_position|    country|birth_date| first_name| last_name|        italian_name|known_name|competition_id|                name|  position|real_position_side|weight|height|team_id|jersey_num|first_nationality|preferred_foot|
+---------+------------------+-----------+----------+-----------+----------+--------------------+----------+--------------+--------------------+----------+------------------+------+------+-------+----------+-----------------+--------------+
|  p225479|           Striker|     Serbia|2000-01-28|      Dusan|  Vlahovic|      dusan vlahovic|      null|            21|      Dusan Vlahovic|   Forward|           Unknown|    75|   190|   t128|         7|           Serbia|          Left|
|   p44898|        Goalkeeper|    Ro

In [ ]:
player_idddsss = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/frames_milan_cagliari.parquet").drop('index')

pre_player_zones = players_locs.join(player_idddsss.select('player_id','player').distinct(), ['player_id']).drop('player_id')
pre_player_zones = pre_player_zones.withColumnRenamed('player','player_id').withColumn('verse', lit(verse))
pd_zones = pre_player_zones.toPandas()

pre_player_zones.show(66)

+--------+------+-------------------+-------------------+------+-----------+----------+-------------+
|    team|  time|                  x|                  y|  type|probability| player_id|        verse|
+--------+------+-------------------+-------------------+------+-----------+----------+-------------+
|Cagliari|4315.0| 23.375560760498047|  9.166890144348145|player|      150.0| Strootman|left_to_right|
|Cagliari|4315.0|  32.42937088012695| 18.611297607421875|player|      150.0|   Dalbert|left_to_right|
|Cagliari|4315.0| 0.7168816924095154|-13.496259689331055|player|      150.0| Pavoletti|left_to_right|
|Cagliari|4315.0| 38.900001525878906| 7.1075568199157715|player|      150.0|   Carboni|left_to_right|
|   Milan|4315.0| 28.276214599609375|  4.420045375823975|player|     -150.0|      Diaz|left_to_right|
|   Milan|4315.0| 18.100000381469727| -8.296259880065918|player|     -150.0|    Krunic|left_to_right|
|Cagliari|4315.0|  47.80747985839844|-2.4706296920776367|player|      150.0| Radun

In [ ]:
def defineLato(x):
  
  y_value = x[3]

  if verse == 'left_to_right':

    if y_value > 16:
      side = 'Destra'
    elif y_value < -16:
      side = 'Sinistra'
    else:
      side = 'Centrale'
  
  else:
    if y_value < -16:
      side = 'Destra'
    elif y_value > 16:
      side = 'Sinistra'
    else:
      side = 'Centrale'

  return side

player_zones = spark.createDataFrame(pre_player_zones.rdd.map(lambda a : list(a)+[defineLato(a)]), pre_player_zones.columns+['lato'])

player_zones.show(50)

+--------+------+-------------------+-------------------+------+-----------+----------+-------------+--------+
|    team|  time|                  x|                  y|  type|probability| player_id|        verse|    lato|
+--------+------+-------------------+-------------------+------+-----------+----------+-------------+--------+
|Cagliari|4315.0| 23.375560760498047|  9.166890144348145|player|      150.0| Strootman|left_to_right|Centrale|
|Cagliari|4315.0|  32.42937088012695| 18.611297607421875|player|      150.0|   Dalbert|left_to_right|  Destra|
|Cagliari|4315.0| 0.7168816924095154|-13.496259689331055|player|      150.0| Pavoletti|left_to_right|Centrale|
|Cagliari|4315.0| 38.900001525878906| 7.1075568199157715|player|      150.0|   Carboni|left_to_right|Centrale|
|   Milan|4315.0| 28.276214599609375|  4.420045375823975|player|     -150.0|      Diaz|left_to_right|Centrale|
|   Milan|4315.0| 18.100000381469727| -8.296259880065918|player|     -150.0|    Krunic|left_to_right|Centrale|
|

In [ ]:
timings_list = timings#list(map(lambda a : a[0], possess.select('time').distinct().collect()))
timings_list

[4315.0]

In [ ]:
player_zones.where(player_zones['team'] == team_a.title()).show(60)

+-----+------+-------------------+-------------------+------+-----------+---------+-------------+--------+
| team|  time|                  x|                  y|  type|probability|player_id|        verse|    lato|
+-----+------+-------------------+-------------------+------+-----------+---------+-------------+--------+
|Milan|4315.0| 28.276214599609375|  4.420045375823975|player|     -150.0|     Diaz|left_to_right|Centrale|
|Milan|4315.0| 18.100000381469727| -8.296259880065918|player|     -150.0|   Krunic|left_to_right|Centrale|
|Milan|4315.0| 37.159332275390625|0.02378525584936142|player|     -150.0|   Giroud|left_to_right|Centrale|
|Milan|4315.0| 18.296260833740234|   5.25933313369751|player|     -150.0| Bennacer|left_to_right|Centrale|
|Milan|4315.0| 1.5630731582641602|0.41248849034309387|player|     -150.0|    Kjaer|left_to_right|Centrale|
|Milan|4315.0| 29.255592346191406| 17.071821212768555|player|     -150.0| Florenzi|left_to_right|  Destra|
|Milan|4315.0|   32.8164176940918|-18

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

def findTimeRoles(time_,player_zones_df,var):
  
  player_zones_t = player_zones_df.where(player_zones_df['time'] == time_).where((player_zones_df['player_id'].isin(['Maignan','Radunovic','Audero']))==False)
  vecAssembler = VectorAssembler(inputCols=[var], outputCol="features")
  new_df_t = vecAssembler.transform(player_zones_t)

  kmeans_t = KMeans(k=3, seed=1)  # 3 clusters here
  model_t = kmeans_t.fit(new_df_t.select('features'))
  
  transformed_t = model_t.transform(new_df_t)

  return transformed_t

player_zones_f_team = findTimeRoles(timings_list[0],player_zones.where(player_zones['team'] == team_a.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
player_zones_ff_team = findTimeRoles(timings_list[0],player_zones_f_team.where(player_zones_f_team['team'] == team_a.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')

#player_zones_f_opp = findTimeRoles(timings_list[0],player_zones.where(player_zones['team'] == opponent.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
#player_zones_ff_opp = findTimeRoles(timings_list[0],player_zones_f_opp.where(player_zones_f_opp['team'] == opponent.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')

for j in timings_list[1:]:
  player_zones_fff_team = findTimeRoles(j,player_zones.where(player_zones['team'] == team_a.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
  player_zones_fff_team_2 = findTimeRoles(j,player_zones_fff_team.where(player_zones_fff_team['team'] == team_a.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')
  player_zones_ff_team = player_zones_ff_team.union(player_zones_fff_team_2)
  #player_zones_fff_opp = findTimeRoles(j,player_zones.where(player_zones['team'] == opponent.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
  #player_zones_fff_opp_2 = findTimeRoles(j,player_zones_fff_opp.where(player_zones_fff_opp['team'] == opponent.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')
  #player_zones_ff_opp = player_zones_ff_opp.union(player_zones_fff_opp_2)

In [ ]:
player_zones_f = player_zones_ff_team#.union(player_zones_ff_opp)

In [ ]:
player_zones_f.where(player_zones_f['team'] == team_a.title()).where(player_zones_f['time'] == 289).orderBy('time','prediction_x','prediction_y').show(70)

+----+----+---+---+----+-----------+---------+-----+----+------------+------------+
|team|time|  x|  y|type|probability|player_id|verse|lato|prediction_x|prediction_y|
+----+----+---+---+----+-----------+---------+-----+----+------------+------------+
+----+----+---+---+----+-----------+---------+-----+----+------------+------------+



In [ ]:
player_zones_team = player_zones_f.where(player_zones_f['team'] == team_a.title())

In [ ]:
avg_lines_side_team = player_zones_team.groupBy('time','prediction_x','prediction_y','verse').agg(func.avg('x'),func.avg('y'))

pd_avg_lines_side_team = avg_lines_side_team.toPandas()

In [ ]:
import itertools

def defineLinesFromAvg(time_comp,pd_avg_lines_side_team):

  lines_per_time = []

  panda_gr_x = pd_avg_lines_side_team[pd_avg_lines_side_team['time']== time_comp].groupby(by='prediction_x').mean('avg(x)').sort_values(by = 'avg(x)')
  lines_x = list(panda_gr_x.index)
  avg_x = panda_gr_x['avg(x)'].values.tolist()
  panda_gr_y = pd_avg_lines_side_team[pd_avg_lines_side_team['time']== time_comp].groupby(by='prediction_y').mean('avg(y)').sort_values(by = 'avg(y)')
  lines_y = list(panda_gr_y.index)
  avg_y = panda_gr_y['avg(y)'].values.tolist()

  if verse == 'left_to_right':
    translate_x = {'0':'Linea Difensiva','1':'Seconda Linea','2':'Terza Linea'}
    translate_y = {'0':'Sinistra','1':'Centrale','2':'Destra'}
  else:
    translate_x = {'2':'Linea Difensiva','1':'Seconda Linea','0':'Terza Linea'}
    translate_y = {'0':'Destra','1':'Centrale','2':'Sinistra'}

  for k in lines_y:
    for j in lines_x: 
      lines_per_time.append([time_comp, j, k, avg_x[lines_x.index(j)], translate_x[str(lines_x.index(j))],avg_y[lines_y.index(k)], translate_y[str(lines_y.index(k))]])
    
  return lines_per_time

In [ ]:
sparkContext=spark.sparkContext
timings_rdd = sparkContext.parallelize(timings)

In [ ]:
lines_definition_per_time = spark.createDataFrame(timings_rdd.flatMap(lambda c : defineLinesFromAvg(c,pd_avg_lines_side_team)), ['time','prediction_x','prediction_y','avg_x','linea','avg_y','lato_new'])

In [ ]:
pre_player_zones_team_final = player_zones_team.drop('lato').join(lines_definition_per_time,['time','prediction_x','prediction_y'])
pre_player_zones_team_final = pre_player_zones_team_final.withColumnRenamed('linea','linea_di_pressione')
player_zones_team_final = pre_player_zones_team_final.withColumnRenamed('lato_new','lato').select('time','linea_di_pressione','lato','avg_x','avg_y').distinct()

In [ ]:
pre_player_zones_team_final.show()

+------+------------+------------+-----+-------------------+-------------------+------+-----------+---------+-------------+-----------------+------------------+-------------------+--------+
|  time|prediction_x|prediction_y| team|                  x|                  y|  type|probability|player_id|        verse|            avg_x|linea_di_pressione|              avg_y|lato_new|
+------+------------+------------+-----+-------------------+-------------------+------+-----------+---------+-------------+-----------------+------------------+-------------------+--------+
|4315.0|           0|           2|Milan| 29.255592346191406| 17.071821212768555|player|     -150.0| Florenzi|left_to_right|30.33519744873047|       Terza Linea| 17.071821212768555|  Destra|
|4315.0|           0|           0|Milan| 28.276214599609375|  4.420045375823975|player|     -150.0|     Diaz|left_to_right|30.33519744873047|       Terza Linea| 3.7177212523917356|Centrale|
|4315.0|           0|           0|Milan| 37.159332

In [ ]:
all_team_lines_and_side = pre_player_zones_team_final.toPandas()

In [ ]:
pd_avg_lines_pivot = player_zones_team_final.toPandas()

In [ ]:
'''
def func_y_value(y):
  
  if y > 11:
    return 'Sinistra'
  elif y < -11:
    return 'Destra'
  else:
    return 'Centrale'
  
def defineLine(t,pre_panda,verse_t,translate_dict,asse,index):
  
  if asse == 'x':
    var = 'prediction_y'
  else:
    var = 'prediction_x'

  out = []

  for i in [0,1,2]:
    try:
      pre_panda_f = pre_panda[pre_panda['time'] == t][pre_panda[var] == i].values.tolist()
      
      if verse_t == 'right_to_left':
        pre_panda_f_sort = sorted(pre_panda_f, key = itemgetter(index), reverse = True)
      else:
        pre_panda_f_sort = sorted(pre_panda_f, key = itemgetter(index))
      
      if asse == 'x':
        pre_panda_f_sort_defined = list(map(lambda c : list(c)+[translate_dict[str(pre_panda_f_sort.index(c))]],pre_panda_f_sort))
      else:
        pre_panda_f_sort_defined = list(map(lambda c : list(c)+[func_y_value(c[index])],pre_panda_f_sort))

      out = out + pre_panda_f_sort_defined
    except:
      pass

  
  return out

translate_x = {'0':'Linea Difensiva','1':'Seconda Linea', '2': 'Terza Linea'}
translate_y = {'0':'Sinistra','1':'Centrale', '2': 'Destra'}

x_index = avg_lines_side_team.columns.index('avg(x)')
y_index = avg_lines_side_team.columns.index('avg(y)')

df_avg_lines_x = spark.createDataFrame(timings_rdd.flatMap(lambda a : defineLine(a,pd_avg_lines_side_team,verse,translate_x,'x',x_index)),['time','cluster_x','cluster_y','verse','avg_x','avg_y','linea_di_pressione'])
df_avg_lines_y = spark.createDataFrame(timings_rdd.flatMap(lambda b : defineLine(b,pd_avg_lines_side_team,verse,translate_y,'y',y_index)),['time','cluster_x','cluster_y','verse','avg_x','avg_y','lato'])

df_avg_lines = df_avg_lines_x.join(df_avg_lines_y.drop('avg_x').drop('avg_y').drop('verse'), ['time','cluster_x','cluster_y'])

df_avg_lines = df_avg_lines.withColumn('avg_y',df_avg_lines['avg_y'].cast(DoubleType())).withColumn('avg_x',df_avg_lines['avg_x'].cast(DoubleType())).orderBy('time','linea_di_pressione','lato')
df_avg_lines = df_avg_lines.withColumn('linea_lato', concat(col('linea_di_pressione'),lit(' '),col('lato')))
df_avg_lines_pivot = df_avg_lines.groupBy('time','verse','lato','linea_di_pressione').agg(func.avg('avg_x'),func.avg('avg_y')).withColumnRenamed('avg(avg_x)','avg_x').withColumnRenamed('avg(avg_y)','avg_y')

df_avg_lines_pivot.show()

pd_avg_lines_pivot = df_avg_lines_pivot.toPandas()
'''

"\ndef func_y_value(y):\n  \n  if y > 11:\n    return 'Sinistra'\n  elif y < -11:\n    return 'Destra'\n  else:\n    return 'Centrale'\n  \ndef defineLine(t,pre_panda,verse_t,translate_dict,asse,index):\n  \n  if asse == 'x':\n    var = 'prediction_y'\n  else:\n    var = 'prediction_x'\n\n  out = []\n\n  for i in [0,1,2]:\n    try:\n      pre_panda_f = pre_panda[pre_panda['time'] == t][pre_panda[var] == i].values.tolist()\n      \n      if verse_t == 'right_to_left':\n        pre_panda_f_sort = sorted(pre_panda_f, key = itemgetter(index), reverse = True)\n      else:\n        pre_panda_f_sort = sorted(pre_panda_f, key = itemgetter(index))\n      \n      if asse == 'x':\n        pre_panda_f_sort_defined = list(map(lambda c : list(c)+[translate_dict[str(pre_panda_f_sort.index(c))]],pre_panda_f_sort))\n      else:\n        pre_panda_f_sort_defined = list(map(lambda c : list(c)+[func_y_value(c[index])],pre_panda_f_sort))\n\n      out = out + pre_panda_f_sort_defined\n    except:\n      pas

In [ ]:
pd_player_zones = player_zones.toPandas()

In [ ]:
player_index = player_zones.columns.index('player_id')
x_index = player_zones.columns.index('x')
y_index = player_zones.columns.index('y')
team_index = player_zones.columns.index('team')
verse_index = player_zones.columns.index('verse') 
time_index = player_zones.columns.index('time') 

player_zones.columns, team_index

(['team',
  'time',
  'x',
  'y',
  'type',
  'probability',
  'player_id',
  'verse',
  'lato'],
 0)

In [ ]:
from numpy.core.function_base import geomspace
def definePositionBetweenLines(x):
  
  player = x[player_index]
  x_value = x[x_index]
  y_value = x[y_index]
  team = x[team_index]
  verse = x[verse_index]
  time = x[time_index]

  pd_time = pd_avg_lines_pivot[pd_avg_lines_pivot['time'] == time]
  pd_player_time = pd_player_zones[pd_player_zones['time'] == time][pd_player_zones['team'] == team_a.title()][pd_player_zones['player_id'] != 'Maignan']

  #terza_linea = float(x[-1])
  
  if verse == 'right_to_left': 
    destra = -33.0
    sinistra = 33.0
    terza_linea = -52.0
  else:
    destra = 33.0
    sinistra = -33.0
    terza_linea = 52.0

  centrale = float(np.mean(pd_time[pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))
  if len(pd_time[pd_time['lato'] == 'Destra']) > 0:
    destra = float(np.mean(pd_time[pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
  if len(pd_time[pd_time['lato'] == 'Sinistra']) > 0:
    sinistra = float(np.mean(pd_time[pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
  
  linea_difensiva = float(np.mean(pd_time[pd_time['linea_di_pressione'] == 'Linea Difensiva']['avg_x'].values.tolist()))
  seconda_linea = float(np.mean(pd_time[pd_time['linea_di_pressione'] == 'Seconda Linea']['avg_x'].values.tolist()))
  if len(pd_time[pd_time['linea_di_pressione'] == 'Terza Linea']) > 0:
    terza_linea = float(np.mean(pd_time[pd_time['linea_di_pressione'] == 'Terza Linea']['avg_x'].values.tolist()))
  else:
    if verse == 'right_to_left':
      most_advanced_player_x = float(np.min(pd_player_time['x'].values.tolist()))
    else:
      most_advanced_player_x = float(np.max(pd_player_time['x'].values.tolist()))
    
    terza_linea = most_advanced_player_x
  
  list_lines = list(map(lambda a : float(np.round(a,1)), [linea_difensiva, seconda_linea,terza_linea]))
  list_lines_lato = list(map(lambda a : float(np.round(a,1)), [centrale,destra,sinistra]))
 
  if team == opponent.title():

    if verse == 'right_to_left':
      
      if x_value >= seconda_linea and x_value <= linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Tra le linee'

      elif x_value > linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))
       
        posizione = 'Profondità'
      
      elif x_value > seconda_linea-3.0 and x_value < seconda_linea:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Possibile Inserimento'

      else:
        posizione = ''
      
      try:
        if y_value < destro:
          lato = 'Ampiezza Destra'
        elif y_value > sinistro:
          lato =  'Ampiezza Sinistra'
        elif y_value <= sinistro and y_value > centrale:
          lato =  'Centro-Sinistra'
        elif y_value >= destro and y_value <= centrale:
          lato =  'Centro-Destra'
        else:
          lato = ''
      except:
        lato = ''
        
    else:
        
      if x_value < seconda_linea and x_value > linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Tra le Linee'
      elif x_value < linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))
       
        posizione = 'Profondità'

      elif x_value < seconda_linea-3.0 and x_value > seconda_linea:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Possibile Inserimento'

      else:
        posizione = ''

      try:
        if y_value > destro:
          lato = 'Ampiezza Destra'
        elif y_value < sinistro:
          lato =  'Ampiezza Sinistra'
        elif y_value >= sinistro and y_value < centrale:
          lato =  'Centro-Sinistra'
        elif y_value <= destro and y_value >= centrale:
          lato =  'Centro-Destra'
        else:
          lato = ''
      except:
        lato = ''

  else:
    if player != 'Maignan':
      posizione,lato = all_team_lines_and_side[all_team_lines_and_side['player_id'] == player][all_team_lines_and_side['time'] == time][['linea_di_pressione','lato_new']].values.tolist()[0]
    else:
      posizione = ''
      lato = ''
  

  out = list(x)+[posizione,lato,list_lines,list_lines_lato]
  return out
  
positioned_players_ = spark.createDataFrame(player_zones.rdd.map(lambda a : definePositionBetweenLines(a)), player_zones.columns+['posizione_x','lato_new','values_x','values_y'])
positioned_players_.where(positioned_players_['time'].isin([1386,40])).orderBy(['time','team','posizione_x','lato']).show(50)

+----+----+---+---+----+-----------+---------+-----+----+-----------+--------+--------+--------+
|team|time|  x|  y|type|probability|player_id|verse|lato|posizione_x|lato_new|values_x|values_y|
+----+----+---+---+----+-----------+---------+-----+----+-----------+--------+--------+--------+
+----+----+---+---+----+-----------+---------+-----+----+-----------+--------+--------+--------+



In [ ]:
positioned_players_r = positioned_players_.drop('lato_new').drop('values_x').drop('values_y')
positioned_players_rr = positioned_players_r#.withColumnRenamed('lato_new','lato')

In [ ]:
positioned_players_out = positioned_players_rr.drop('type')

In [ ]:
positioned_players_out.where(positioned_players_out['time'] == 1386).orderBy('time','team','posizione_x','lato').show(22)

+----+----+---+---+-----------+---------+-----+----+-----------+
|team|time|  x|  y|probability|player_id|verse|lato|posizione_x|
+----+----+---+---+-----------+---------+-----+----+-----------+
+----+----+---+---+-----------+---------+-----+----+-----------+



In [ ]:
def addNegativeProb(x):

  team = x[team_index] 
  probability = x[prob_index]

  if team == team_a.title(): 
    new_prob = -float(probability) 
  else: 
    new_prob = float(probability)

  y = list(x[:prob_index])+[new_prob]+list(x[1+prob_index:])

  return y

pitch_pivot_union = pitch_pivot.distinct()

team_index, prob_index = pitch_pivot_union.columns.index('team'), pitch_pivot_union.columns.index('probability')

pitch_pivot_union = spark.createDataFrame(pitch_pivot_union.rdd.map(lambda y : addNegativeProb(y)), pitch_pivot_union.columns)

for i in players_locs.columns: 
  if 'class' in str(i): 
    pitch_pivot_union = pitch_pivot_union.withColumn(str(i), lit(''))

out = pitch_pivot_union.union(players_locs.select(pitch_pivot_union.columns)).union(ball_locs.select(pitch_pivot_union.columns)).withColumnRenamed('player_id','player')  

In [ ]:
pd_pitch = max_pitch_pivot_jj.toPandas()

In [ ]:
panda_possess = possess.toPandas()
panda_possess_x_y = out.where(out.team != team_a.title()).where(out.team != opponent.title()).toPandas()

In [ ]:
from bisect import bisect
from pyspark.sql.window import Window

def areaDanger(d,x_ranges,y_ranges):
  
  if d[verse_index] == 'left_to_right':
    x_values = x_ranges_value_right_to_left
  else:
    x_values = x_ranges_value_left_to_right
  
  try:
    x_value = x_values[bisect(x_ranges,d[x_index])]
  except:
    try:
      x_value = x_values[bisect(x_ranges,d[x_index])-1]
    except:
      x_value = x_values[bisect(x_ranges,d[x_index])+1]
      
  try:
    y_value = y_ranges_value[bisect(y_ranges,d[y_index])]
  except:
    try:
      y_value = y_ranges_value[bisect(y_ranges,d[y_index])-1]
    except:
      y_value = y_ranges_value[bisect(y_ranges,d[y_index])+1]
  
  if x_value < 50:
    y_value = 10
  
  return [bisect(x_ranges,d[x_index]),bisect(y_ranges,d[y_index]),float(np.round(x_value,0)),float(np.round(y_value,0)),float(np.round(x_value+y_value,0))]
 
def findOpponentsOvercomeByThePass(x,panda,panda_possess,panda_possess_bis,pd_pitch):
  
  player_x = x[x_index]
  player_y = x[y_index]
  verse = x[verse_index]
  time_ = x[time_index]
  
  player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0]
  possess_x,possess_y = panda_possess_bis[panda_possess_bis['time'] == time_][['x','y']].values.tolist()[0]
  
  sit_x = player_x
  sit_y = player_y

  x_advancement_flag = float(np.sign(team_a_goal_line[0] - player_x))

  values_x = [sit_x-(x_advancement_flag*10),sit_x+(x_advancement_flag*10)]

  back_min_x,ahead_max_x = float(np.min(values_x)),float(np.max(values_x))

  up_y = sit_y+10
  down_y = sit_y-10

  quarters_control_percentages = []

  close_opponents_panda = panda[panda['team'] == team_a.title()][panda['time'] == time_][panda['x'] > player_x-4.0][panda['x'] < player_x+4.0][panda['y'] > player_y-4.0][panda['x'] < player_y+4.0]
  close_opponents = len(close_opponents_panda)
  opponents_x_y = panda[panda['team'] == team_a.title()][panda['time'] == time_][panda['player_id'] != team_a_goalies][['x','y']].values.tolist()
  
  #player_distance_from_goal_line = float(np.sqrt((player_x - team_a_goal_line[0])**2 + (player_y - team_a_goal_line[1])**2))
  #possess_distance_from_goal_line = float(np.sqrt((possess_x - team_a_goal_line[0])**2 + (possess_y - team_a_goal_line[1])**2))

  if verse == 'left_to_right':
    player_x_distance_from_goal_line = float(player_x - team_a_goal_line[0])
    possess_x_distance_from_goal_line = float(possess_x - team_a_goal_line[0])
  else:
    player_x_distance_from_goal_line = -float(player_x - team_a_goal_line[0])
    possess_x_distance_from_goal_line = -float(possess_x - team_a_goal_line[0])
  
  team_advancement_x = possess_x_distance_from_goal_line - player_x_distance_from_goal_line
  ball_distance = float(np.sqrt((player_x-possess_x)**2+(player_y-possess_y)**2))

  #opponents_distances_to_goal_line = []
  opponents_distances_to_goal_line_x = []

  for j in opponents_x_y:
    if verse == 'left_to_right':
      dist = -float(team_a_goal_line[0]-j[0])
    else:
      dist = float(team_a_goal_line[0]-j[0])

    #opponents_distances_to_goal_line.append(float(np.sqrt((team_a_goal_line[0]-j[0])**2+(team_a_goal_line[1]-j[1])**2)))
    opponents_distances_to_goal_line_x.append(dist)
    
  #opponents_from_goal_line = len(list(filter(lambda a : a < player_distance_from_goal_line-1.0, opponents_distances_to_goal_line)))
  #opponents_from_possess_to_goal_line = len(list(filter(lambda a : a < possess_distance_from_goal_line-1.0, opponents_distances_to_goal_line)))

  opponents_from_goal_line_x = len(list(filter(lambda a : a < player_x_distance_from_goal_line-4.0, opponents_distances_to_goal_line_x)))
  opponents_from_possess_to_goal_line_x = len(list(filter(lambda a : a < possess_x_distance_from_goal_line-1.0, opponents_distances_to_goal_line_x)))

  #opponents_cut = opponents_from_possess_to_goal_line-opponents_from_goal_line
  opponents_cut_x = opponents_from_possess_to_goal_line_x-opponents_from_goal_line_x

  #return [closest_opponent_distance,opponents_from_goal_line,close_opponents,player_distance_from_goal_line,opponents_cut]
  return [opponents_from_possess_to_goal_line_x,close_opponents,player_x_distance_from_goal_line,opponents_cut_x,team_advancement_x,ball_distance]
  
pre_kpis = max_pitch_pivot_jjj.where((max_pitch_pivot_jjj['player_id'].isin([team_a_goalies])) == False).select('player_id','team','verse','time','x','y').distinct()

verse_index = pre_kpis.columns.index('verse')
time_index = pre_kpis.columns.index('time')
y_index = pre_kpis.columns.index('y')
x_index = pre_kpis.columns.index('x')

pre_kpis = pre_kpis.withColumn('x', pre_kpis['x'].cast(DoubleType())).withColumn('y', pre_kpis['y'].cast(DoubleType()))

ppp_area_targets = spark.createDataFrame(pre_kpis.rdd.map(lambda g : list(g)+areaDanger(g,x_ranges,y_ranges)+findOpponentsOvercomeByThePass(g,panda_locs,panda_possess,panda_possess_x_y,pd_pitch)), pre_kpis.columns+['x_area','y_area','x_value','y_value','area_value','possess_opponents_from_goal_line','close_opponents','player_distance_from_goal_line','opponents_cut','team_advancement','ball_distance'])


In [ ]:
area_targets_advance_pre_final = ppp_area_targets.select('player_id','time','team','area_value','ball_distance','possess_opponents_from_goal_line','x','y','close_opponents','player_distance_from_goal_line','opponents_cut','team_advancement','verse').distinct()

area_targets_advance_pre_final.show()

+---------+------+--------+----------+------------------+--------------------------------+------------------+-------------------+---------------+------------------------------+-------------+------------------+-------------+
|player_id|  time|    team|area_value|     ball_distance|possess_opponents_from_goal_line|                 x|                  y|close_opponents|player_distance_from_goal_line|opponents_cut|  team_advancement|        verse|
+---------+------+--------+----------+------------------+--------------------------------+------------------+-------------------+---------------+------------------------------+-------------+------------------+-------------+
|   Nandez|4315.0|Cagliari|      30.0|53.469718261046346|                              11|31.888702392578125|-14.020622253417969|              0|             87.08870315551758|            4|23.311298370361328|left_to_right|
|    Godin|4315.0|Cagliari|      23.0| 47.37781679830757|                              11|44.62752532958

In [ ]:
def singlePointOpponents(x,y,panda_locs,opponent,time_,meters_to_intercept,possess_x,possess_y,fromPossessOrToGoal):
  
  distance = distance = float(np.sqrt(((x-possess_x)**2)+((y-possess_y)**2)))
  
  if fromPossessOrToGoal != 'possess':
    opponent_meters_run = 3.0*(float(distance)/8.0)
  else:
    opponent_meters_run = 2.0*(float(distance)/20.0)
  
  opps =  panda_locs[panda_locs['team'] == team_a.title()][panda_locs['player_id'] != 'Maignan'] #automatizzare -> team_a_goalies è in codice (p123...)
  on_trajectory = opps[opps['time'] == time_][opps['x'] > x-meters_to_intercept-opponent_meters_run][opps['x'] < x+meters_to_intercept+opponent_meters_run][opps['y'] > y-meters_to_intercept-opponent_meters_run][opps['y'] < y+meters_to_intercept+opponent_meters_run]['player_id'].values.tolist()
 
  return on_trajectory

def interceptionOpponentsFix(player,time_,target_x,target_y,verse,panda,panda_possess,panda_possess_x_y,fromPossessOrToGoal):
  
  if fromPossessOrToGoal == 'possess':
    player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0]
    possess_x,possess_y = panda_possess_x_y[panda_possess_x_y['time'] == time_][['x','y']].values.tolist()[0]
  else:
    player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0][0]
    possess_x,possess_y = team_a_goal_line[0], team_a_goal_line[1]
  
  distance = float(np.sqrt(((target_x-possess_x)**2)+((target_y-possess_y)**2)))

  m = (possess_y-target_y)/(possess_x-target_x)

  b = (possess_x*target_y - target_x*possess_y)/(possess_x-target_x)

  if possess_x > target_x:
    xs = sorted(list(np.arange(target_x,possess_x,float(possess_x-target_x)/distance)), reverse = True)
  else:
    xs = list(np.arange(possess_x,target_x,float(target_x-possess_x)/distance))

  pre_trajectory = list(map(lambda xx: [float(xx),float((m*xx)+b)], xs))

  if fromPossessOrToGoal == 'possess':
    trajectory = pre_trajectory[:10]
    end_trajectory = pre_trajectory[-10:]
    interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,possess_x,possess_y,fromPossessOrToGoal), trajectory))))
    interception_opponents_no_duplicates = len(list(dict.fromkeys(interception_opponents)))
  else:
    trajectory = pre_trajectory[:10]
    interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,target_x,target_y,fromPossessOrToGoal), trajectory))))
    interception_opponents_no_duplicates = len(list(interception_opponents))

  if str(player) == str(player_possess):
    possessFlag = "Sul portatore"
  else:
    possessFlag = ""
  
  pre_interception_opponents = [interception_opponents_no_duplicates]
  
  if fromPossessOrToGoal == 'possess':
    close_opponents_distances = []
    if verse == 'right_to_left':
      opps_data = panda[panda['time'] == time_][panda['team'] == team_a.title()][panda['x'] > target_x-2.0][['player_id','x','y']].values.tolist()
    else:
      opps_data = panda[panda['time'] == time_][panda['team'] == team_a.title()][panda['x'] < target_x+2.0][['player_id','x','y']].values.tolist()
  
    for j in opps_data:
     close_opponents_distances.append([j[0],float(np.sqrt((j[1]-target_x)**2+(j[2]-target_y)**2))])
    
    closest_opponent_player, closest_opponent_distance = sorted(close_opponents_distances, key=itemgetter(1))[0]
  
    aa_interception_opponents = pre_interception_opponents+[possess_x,possess_y,possessFlag,closest_opponent_distance,closest_opponent_player,' '.join(list(dict.fromkeys(interception_opponents)))]
  else:
    aa_interception_opponents = pre_interception_opponents+[' '.join(list(dict.fromkeys(list(map(lambda a : str(a[:3]),interception_opponents)))))]
  
  return aa_interception_opponents

In [ ]:
def singlePointOpponents(x,y,panda_locs,opponent,time_,meters_to_intercept,possess_x,possess_y,fromPossessOrToGoal):
  
  distance = distance = float(np.sqrt(((x-possess_x)**2)+((y-possess_y)**2)))
  
  if fromPossessOrToGoal != 'possess':
    opponent_meters_run = 3.5*(float(distance)/8.0)
  else:
    opponent_meters_run = 2.0*(float(distance)/20.0)
  opps =  panda_locs[panda_locs['team'] == team_a.title()][panda_locs['player_id'] != 'Maignan'] #automatizzare -> team_a_goalies è in codice (p123...)
  on_trajectory = opps[opps['time'] == time_][opps['x'] > x-meters_to_intercept-opponent_meters_run][opps['x'] < x+meters_to_intercept+opponent_meters_run][opps['y'] > y-meters_to_intercept-opponent_meters_run][opps['y'] < y+meters_to_intercept+opponent_meters_run]['player_id'].values.tolist()
 
  return on_trajectory


def interceptionOpponentsFix(player,time_,target_x,target_y,verse,panda,panda_possess,panda_possess_x_y,fromPossessOrToGoal):
  
  if fromPossessOrToGoal == 'possess':
    player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0]
    possess_x,possess_y = panda_possess_x_y[panda_possess_x_y['time'] == time_][['x','y']].values.tolist()[0]
  else:
    player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0][0]
    possess_x,possess_y = team_a_goal_line[0], team_a_goal_line[1]
  
  distance = float(np.sqrt(((target_x-possess_x)**2)+((target_y-possess_y)**2)))

  m = (possess_y-target_y)/(possess_x-target_x)

  b = (possess_x*target_y - target_x*possess_y)/(possess_x-target_x)

  if possess_x > target_x:
    xs = sorted(list(np.arange(target_x,possess_x,float(possess_x-target_x)/distance)), reverse = True)
  else:
    xs = list(np.arange(possess_x,target_x,float(target_x-possess_x)/distance))

  pre_trajectory = list(map(lambda xx: [float(xx),float((m*xx)+b)], xs))

  if fromPossessOrToGoal == 'possess':
    trajectory = pre_trajectory[:10]
    end_trajectory = pre_trajectory[-10:]
    interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,possess_x,possess_y,fromPossessOrToGoal), trajectory))))
    interception_opponents_no_duplicates_value = len(list(interception_opponents))
    interception_opponents_no_duplicates_players = len(list(dict.fromkeys(interception_opponents)))
    end_interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,possess_x,possess_y,fromPossessOrToGoal), end_trajectory))))
    end_interception_opponents_no_duplicates_value = len(list(end_interception_opponents))
    end_interception_opponents_no_duplicates_players = len(list(dict.fromkeys(end_interception_opponents)))
  else:
    trajectory = pre_trajectory[:10]
    interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,target_x,target_y,fromPossessOrToGoal), trajectory))))
    interception_opponents_no_duplicates_value = len(list(interception_opponents))
    interception_opponents_no_duplicates_players = len(list(dict.fromkeys(interception_opponents)))
    end_interception_opponents_no_duplicates_value = 0
    end_interception_opponents_no_duplicates_players = ''
  if str(player) == str(player_possess):
    possessFlag = "Sul portatore"
  else:
    possessFlag = ""
  
  pre_interception_opponents = [interception_opponents_no_duplicates_value,interception_opponents_no_duplicates_players,end_interception_opponents_no_duplicates_value,end_interception_opponents_no_duplicates_players]
  
  if fromPossessOrToGoal == 'possess':
    close_opponents_distances = []
    if verse == 'right_to_left':
      opps_data = panda[panda['time'] == time_][panda['team'] == team_a.title()][panda['x'] > target_x-2.0][['player_id','x','y']].values.tolist()
    else:
      opps_data = panda[panda['time'] == time_][panda['team'] == team_a.title()][panda['x'] < target_x+2.0][['player_id','x','y']].values.tolist()
  
    for j in opps_data:
     close_opponents_distances.append([j[0],float(np.sqrt((j[1]-target_x)**2+(j[2]-target_y)**2))])
    
    closest_opponent_player, closest_opponent_distance = sorted(close_opponents_distances, key=itemgetter(1))[0]
  
    aa_interception_opponents = pre_interception_opponents+[possess_x,possess_y,possessFlag,closest_opponent_distance,closest_opponent_player]
  else:
    aa_interception_opponents = pre_interception_opponents
  
  return aa_interception_opponents

In [ ]:
interception_opps = spark.createDataFrame(area_targets_advance_pre_final.select('player_id','time','x','y','verse').distinct().rdd.map(lambda g : list(g)+list(interceptionOpponentsFix(g[0],g[1],g[2],g[3],g[4],panda_locs,panda_possess,panda_possess_x_y,'possess'))+list(interceptionOpponentsFix(g[0],g[1],g[2],g[3],g[4],panda_locs,panda_possess,panda_possess_x_y,'goal'))), ['player_id','time','x','y','verse','interception_opponents_value','interception_opponents_players','schermo_opponents_value','schermo_opponents_players','possess_goal_x','possess_goal_y','tipo_di_marcatura','closest_opponent_distance','closest_opponent_player','interception_players','opponents_to_goal_line','to_goal_line_players'])

In [ ]:
interception_opps.where(interception_opps['time'] == 1386).show()

+---------+----+---+---+-----+----------------------------+------------------------------+-----------------------+-------------------------+--------------+--------------+-----------------+-------------------------+-----------------------+--------------------+----------------------+--------------------+---+
|player_id|time|  x|  y|verse|interception_opponents_value|interception_opponents_players|schermo_opponents_value|schermo_opponents_players|possess_goal_x|possess_goal_y|tipo_di_marcatura|closest_opponent_distance|closest_opponent_player|interception_players|opponents_to_goal_line|to_goal_line_players|_18|
+---------+----+---+---+-----+----------------------------+------------------------------+-----------------------+-------------------------+--------------+--------------+-----------------+-------------------------+-----------------------+--------------------+----------------------+--------------------+---+
+---------+----+---+---+-----+----------------------------+-----------------

In [ ]:
#count e probability

count_and_probs = pitch_pivot_union.groupBy('time','player_id').agg(func.count('time'),func.avg('probability')).withColumnRenamed('count(time)','Area (Mq)').withColumnRenamed('avg(probability)','Probabilità di Ricezione')
count_and_probs.where(count_and_probs['time'].isin([289,1386])).orderBy('time').show(50)

+----+---------+---------+------------------------+
|time|player_id|Area (Mq)|Probabilità di Ricezione|
+----+---------+---------+------------------------+
+----+---------+---------+------------------------+



In [ ]:
duelli = positioned_players_out.join(count_and_probs,['time','player_id'])

In [ ]:
duelli.where(duelli['time'] == 1386).orderBy('team','posizione_x','lato').show()

+----+---------+----+---+---+-----------+-----+----+-----------+---------+------------------------+
|time|player_id|team|  x|  y|probability|verse|lato|posizione_x|Area (Mq)|Probabilità di Ricezione|
+----+---------+----+---+---+-----------+-----+----+-----------+---------+------------------------+
+----+---------+----+---+---+-----------+-----+----+-----------+---------+------------------------+



In [ ]:
kpis = interception_opps.join(count_and_probs, ['time','player_id'])
kpis.show()

+------+---------+------------------+-------------------+-------------+----------------------------+------------------------------+-----------------------+-------------------------+-----------------+------------------+-----------------+-------------------------+-----------------------+--------------------+----------------------+--------------------+---+---------+------------------------+
|  time|player_id|                 x|                  y|        verse|interception_opponents_value|interception_opponents_players|schermo_opponents_value|schermo_opponents_players|   possess_goal_x|    possess_goal_y|tipo_di_marcatura|closest_opponent_distance|closest_opponent_player|interception_players|opponents_to_goal_line|to_goal_line_players|_18|Area (Mq)|Probabilità di Ricezione|
+------+---------+------------------+-------------------+-------------+----------------------------+------------------------------+-----------------------+-------------------------+-----------------+------------------+

In [ ]:
kpis_out = area_targets_advance_pre_final.drop('x').drop('y').join(kpis.drop('verse'),['time','player_id'])
kpis_outtt = kpis_out.drop('_18')#.join(timed_roles, ['time','player_id'])

kpis_outtt.show()

+------+---------+--------+----------+------------------+--------------------------------+---------------+------------------------------+-------------+------------------+-------------+------------------+-------------------+----------------------------+------------------------------+-----------------------+-------------------------+-----------------+------------------+-----------------+-------------------------+-----------------------+--------------------+----------------------+--------------------+---------+------------------------+
|  time|player_id|    team|area_value|     ball_distance|possess_opponents_from_goal_line|close_opponents|player_distance_from_goal_line|opponents_cut|  team_advancement|        verse|                 x|                  y|interception_opponents_value|interception_opponents_players|schermo_opponents_value|schermo_opponents_players|   possess_goal_x|    possess_goal_y|tipo_di_marcatura|closest_opponent_distance|closest_opponent_player|interception_players|op

In [ ]:
'''
# area_value
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('area_value'),func.max('area_value')).withColumnRenamed('avg(area_value)','avg_area_value').withColumnRenamed('max(area_value)','max').withColumn('kpi', lit('area_value')).fillna(0.0)
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final_area_value.withColumn('area_value_index', (0.45*area_targets_advance_pre_final_area_value['avg_area_value']+0.55*area_targets_advance_pre_final_area_value['max'])).select('player_id','time','avg_area_value','area_value_index')
# opponents_cut
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('opponents_cut'),func.max('opponents_cut')).withColumnRenamed('avg(opponents_cut)','avg_opponents_cut').withColumnRenamed('max(opponents_cut)','max').withColumn('kpi', lit('opponents_cut')).fillna(0.0)
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final_opponents_cut.withColumn('opponents_cut_index', (0.45*area_targets_advance_pre_final_opponents_cut['avg_opponents_cut']+0.55*area_targets_advance_pre_final_opponents_cut['max'])).select('player_id','time','avg_opponents_cut','opponents_cut_index')

# percentage_zone_in_control
#area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('percentage_zone_in_control'),func.max('percentage_zone_in_control')).withColumnRenamed('avg(percentage_zone_in_control)','avg').withColumnRenamed('max(percentage_zone_in_control)','max').withColumn('kpi', lit('percentage_zone_in_control'))
#area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final_percentage_zone_in_control.withColumn('percentage_zone_in_control_index', func.log((1+(0.45*area_targets_advance_pre_final_percentage_zone_in_control['avg']+0.55*area_targets_advance_pre_final_percentage_zone_in_control['max']))/area_targets_advance_pre_final_percentage_zone_in_control['avg'])).select('player_id','time','percentage_zone_in_control_index')

# team_advancement
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('team_advancement'),func.max('team_advancement')).withColumnRenamed('avg(team_advancement)','avg_team_advancement').withColumnRenamed('max(team_advancement)','max').withColumn('kpi', lit('team_advancement')).fillna(0.0)
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final_team_advancement.withColumn('team_advancement_index', (0.45*area_targets_advance_pre_final_team_advancement['avg_team_advancement']+0.55*area_targets_advance_pre_final_team_advancement['max'])).select('player_id','time','avg_team_advancement','team_advancement_index')
# distance_from_target
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('distance_from_target'),func.max('distance_from_target')).withColumnRenamed('avg(distance_from_target)','avg_distance_from_target').withColumnRenamed('max(distance_from_target)','max').withColumn('kpi', lit('distance_from_target')).fillna(0.0)
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final_distance_from_target.withColumn('distance_from_target_index', (0.45*area_targets_advance_pre_final_distance_from_target['avg_distance_from_target']+0.55*area_targets_advance_pre_final_distance_from_target['max'])).select('player_id','time','distance_from_target_index')
# probability
area_targets_advance_pre_final_probability = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('probability'),func.max('probability')).withColumnRenamed('avg(probability)','avg_probability').withColumnRenamed('max(probability)','max').withColumn('kpi', lit('probability')).fillna(0.0)
area_targets_advance_pre_final_probability = area_targets_advance_pre_final_probability.withColumn('probability_index', (0.45*area_targets_advance_pre_final_probability['avg_probability']+0.55*area_targets_advance_pre_final_probability['max'])).select('player_id','time','avg_probability','probability_index')
# ball_distance
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('ball_distance'),func.max('ball_distance')).withColumnRenamed('avg(ball_distance)','avg_ball_distance').withColumnRenamed('max(ball_distance)','max').withColumn('kpi', lit('ball_distance')).fillna(0.0)
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final_ball_distance.withColumn('ball_distance_index', (0.45*area_targets_advance_pre_final_ball_distance['avg_ball_distance']+0.55*area_targets_advance_pre_final_ball_distance['max'])).select('player_id','time','avg_ball_distance','ball_distance_index')

count = area_targets_advance_pre_final.groupBy('player_id','time').count()

kpis = area_targets_advance_pre_final_area_value.join(count,['player_id','time']).join(area_targets_advance_pre_final_opponents_cut, ['player_id','time']).join(area_targets_advance_pre_final_team_advancement, ['player_id','time']).join(area_targets_advance_pre_final_distance_from_target, ['player_id','time']).join(area_targets_advance_pre_final_probability, ['player_id','time']).join(area_targets_advance_pre_final_ball_distance, ['player_id','time'])
'''

"\n# area_value\narea_targets_advance_pre_final_area_value = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('area_value'),func.max('area_value')).withColumnRenamed('avg(area_value)','avg_area_value').withColumnRenamed('max(area_value)','max').withColumn('kpi', lit('area_value')).fillna(0.0)\narea_targets_advance_pre_final_area_value = area_targets_advance_pre_final_area_value.withColumn('area_value_index', (0.45*area_targets_advance_pre_final_area_value['avg_area_value']+0.55*area_targets_advance_pre_final_area_value['max'])).select('player_id','time','avg_area_value','area_value_index')\n# opponents_cut\narea_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('opponents_cut'),func.max('opponents_cut')).withColumnRenamed('avg(opponents_cut)','avg_opponents_cut').withColumnRenamed('max(opponents_cut)','max').withColumn('kpi', lit('opponents_cut')).fillna(0.0)\narea_targets_advance_pre_final_opponents

In [ ]:
duelli_path = 'duelli_'+name_path
duelli_path

'duelli_milan_cagliari_second_half_2_all'

In [ ]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+duelli_path+".csv"

duelli.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)

True

In [ ]:
new_path = 'pitch_control_2_'+name_path
new_path

'pitch_control_2_milan_cagliari_second_half_2_all'

In [ ]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+new_path+".csv"

out.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)

In [ ]:
new_path_k = 'kpis_new_'+name_path
new_path_k

In [ ]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp_k"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+new_path_k+".csv"

kpis_out.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)